In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 데이터셋 클래스 정의
class ConditionalDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.summer_images = os.listdir(os.path.join(root_dir, 'train_summer'))
        self.winter_images = os.listdir(os.path.join(root_dir, 'train_winter'))

    def __len__(self):
        return max(len(self.summer_images), len(self.winter_images))

    def __getitem__(self, idx):
        if idx < len(self.summer_images):
            img_name = os.path.join(self.root_dir, 'train_summer', self.summer_images[idx])
            label = 0  # 여름은 라벨 0
        else:
            img_name = os.path.join(self.root_dir, 'train_winter', self.winter_images[idx % len(self.winter_images)])
            label = 1  # 겨울은 라벨 1
        image = Image.open(img_name)
        if self.transform:
        # image를 키워드 인자로 전달
            image = np.array(image)  # albumentations는 PIL Image 대신 numpy array를 사용
            image = self.transform(image=image)['image']
        return image, label



# albumentations를 사용한 전처리 정의
transform = A.Compose(
    [
        A.Resize(256, 256),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        ToTensorV2(),
    ]
)

In [4]:
# 설정값
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
epochs = 200
nz = 100  # 노이즈 차원
lr = 0.0002
beta1 = 0.5

In [5]:
class Generator(nn.Module):
    def __init__(self, nz, num_classes):
        super(Generator, self).__init__()
        self.nz = nz
        self.main = nn.Sequential(
            # nz: 잠재 벡터의 크기
            # 여기서는 더 깊은 네트워크 구조를 가정
            nn.Linear(self.nz, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 128 * 64 * 64),
            nn.ReLU(True),
            nn.BatchNorm1d(128 * 64 * 64),
            nn.Unflatten(1, (128, 64, 64)),
            # nn.ConvTranspose2d를 사용하여 이미지를 점진적으로 업샘플링
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 출력: 128 x 128
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 출력: 256 x 256
            nn.ReLU(True),
            nn.ConvTranspose2d(
                32, 3, 3, stride=1, padding=1
            ),  # 출력 채널을 이미지의 채널 수에 맞게 조정
            nn.Tanh(),  # 이미지의 픽셀 값은 -1과 1 사이
        )
        self.label_emb = nn.Embedding(num_classes, nz)

    def forward(self, x, labels):
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)  # 잠재 벡터 x와 조건 c를 연결
        return self.main(x)

In [6]:
class Discriminator(nn.Module):
    def __init__(self, num_classes):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # 입력 이미지 크기에 맞게 조정
            nn.Conv2d(3, 64, 4, stride=2, padding=1),  # 입력: 256 x 256
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 128 x 128
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),  # 64 x 64
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, stride=2, padding=1),  # 32 x 32
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),  # 16 x 16
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, stride=1, padding=0),  # 13 x 13
            nn.Flatten(),
            nn.Linear(13 * 13, 1),
            nn.Sigmoid(),
        )
        self.label_emb = nn.Embedding(num_classes, 256 * 256 * 3)


    def forward(self, x, labels):
        # 라벨 임베딩을 이미지 텐서와 같은 차원으로 확장
        c = self.label_emb(labels)  # 이 부분은 [batch_size, embed_size] 차원을 가짐
        c = c.unsqueeze(2).unsqueeze(3)  # [batch_size, embed_size, 1, 1] 형태로 만듦
        c = c.repeat(1, 1, x.size(2), x.size(3))  # [batch_size, embed_size, height, width] 차원으로 확장
        x = torch.cat([x, c], 1)  # 이미지 텐서 x와 라벨 임베딩 c를 연결
        return self.main(x)

In [7]:
# 데이터셋과 데이터 로더 설정
train_dataset = ConditionalDataset(
    root_dir='/content/drive/MyDrive/CDAL/summer2winter_yosemite/train', # 또는 train_winter
    transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [8]:
# 모델 초기화
G = Generator(nz, num_classes=2).to(device)
D = Discriminator(num_classes=2).to(device)

# 손실 함수와 최적화 함수
criterion = nn.BCELoss()
optimizerD = optim.Adam(D.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(G.parameters(), lr=lr, betas=(beta1, 0.999))

# 폴더 생성
os.makedirs('generated_images', exist_ok=True)


In [ ]:

# 학습
for epoch in range(epochs):
    for i, (images, labels) in enumerate(tqdm(train_loader)):
        # 실제 데이터에 대한 판별자의 손실 계산
        D.zero_grad()
        real_images = images.to(device)
        b_size = real_images.size(0)
        real_label = torch.full((b_size,), 1, device=device)
        labels = labels.to(device)

        output = D(real_images, labels).view(-1)
        errD_real = criterion(output, real_label)
        errD_real.backward()
        D_x = output.mean().item()

        # 가짜 데이터에 대한 판별자의 손실 계산
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake_images = G(noise, labels)
        fake_label = torch.full((b_size,), 0, device=device)
        output = D(fake_images.detach(), labels).view(-1)
        errD_fake = criterion(output, fake_label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        # 생성자의 손실 계산
        G.zero_grad()
        output = D(fake_images, labels).view(-1)
        errG = criterion(output, real_label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # 진행 상황 출력 및 이미지 저장
        if i % 50 == 0:
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                  % (epoch, epochs, i, len(train_loader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
            save_image(fake_images, '/content/drive/MyDrive/CDAL/generated_images/epoch_%03d.png' % epoch)


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:

# 모델 저장
torch.save(G.state_dict(), 'generator.pth')
torch.save(D.state_dict(), 'discriminator.pth')